In [1]:
# notebooks/03-feature-engineering.ipynb

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# --- Kết nối tới Database (tương tự notebook trước) ---
load_dotenv()
db_user = os.getenv('MYSQL_USER')
db_password = os.getenv('MYSQL_PASSWORD')
db_host = os.getenv('MYSQL_HOST')
db_port = os.getenv('MYSQL_PORT')
db_name = os.getenv('MYSQL_DATABASE')
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection_string)

# --- Viết Master Query ---
# Câu lệnh này sẽ JOIN hầu hết các bảng quan trọng lại với nhau
master_query = """
SELECT
    o.order_id,
    -- Target variable
    r.review_score,
    
    -- Features từ order_items
    oi.price,
    oi.freight_value,
    
    -- Features từ orders
    o.order_status,
    o.order_purchase_timestamp,
    o.order_approved_at,
    o.order_delivered_customer_date,
    o.order_estimated_delivery_date,
    
    -- Features từ products
    p.product_category_name,
    p.product_weight_g,
    p.product_length_cm,
    p.product_height_cm,
    p.product_width_cm,
    
    -- Features từ sellers
    s.seller_state,
    
    -- Features từ customers
    c.customer_state
FROM
    orders o
JOIN order_reviews r ON o.order_id = r.order_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
JOIN sellers s ON oi.seller_id = s.seller_id
JOIN customers c ON o.customer_id = c.customer_id
WHERE
    o.order_status = 'delivered' AND o.order_delivered_customer_date IS NOT NULL;
"""

# Thực thi query và load vào DataFrame
print("Đang tải dữ liệu từ Master Query...")
df = pd.read_sql(master_query, engine)
print(f"Tải thành công {len(df)} dòng dữ liệu.")
display(df.head())

Đang tải dữ liệu từ Master Query...
Tải thành công 110005 dòng dữ liệu.


,order_id,review_score,price,freight_value,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_state,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,5,58.90,13.29,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-20 23:43:48,2017-09-29,cool_stuff,650.0,28.0,9.0,14.0,SP,RJ
1,00018f77f2f0320c557190d7a144bdd3,4,239.90,19.93,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-12 16:04:24,2017-05-15,pet_shop,30000.0,50.0,30.0,40.0,SP,SP
2,000229ec398224ef6ca0657da4fc703e,5,199.00,17.87,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-22 13:19:16,2018-02-05,moveis_decoracao,3050.0,33.0,13.0,33.0,MG,MG
3,00024acbcdf0a6daa1e931b038114c75,4,12.99,12.79,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-14 13:32:39,2018-08-20,perfumaria,200.0,16.0,10.0,15.0,SP,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,5,199.90,18.14,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-03-01 16:42:31,2017-03-17,ferramentas_jardim,3750.0,35.0,40.0,30.0,PR,SP


In [2]:
# Chuyển đổi các cột ngày tháng sang kiểu datetime
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
for col in date_cols:
    df[col] = pd.to_datetime(df[col])

# --- Tạo các feature về thời gian vận chuyển ---

# 1. Thời gian giao hàng thực tế (ngày)
df['delivery_days'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.total_seconds() / (24 * 3600)

# 2. Thời gian giao hàng dự kiến (ngày)
df['estimated_delivery_days'] = (df['order_estimated_delivery_date'] - df['order_purchase_timestamp']).dt.total_seconds() / (24 * 3600)

# 3. Độ trễ so với dự kiến (ngày) -> Feature cực kỳ quan trọng!
# Số dương nghĩa là giao trễ, số âm là giao sớm
df['delivery_delay'] = df['delivery_days'] - df['estimated_delivery_days']

# 4. Feature nhị phân: có giao trễ hay không?
df['is_late'] = (df['delivery_delay'] > 0).astype(int)

# --- Tạo các feature khác ---

# 5. Tỷ lệ phí vận chuyển trên giá sản phẩm
# Thêm 1e-6 để tránh chia cho 0
df['freight_ratio'] = df['freight_value'] / (df['price'] + 1e-6)

# 6. Kích thước thể tích sản phẩm
df['product_volume_cm3'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

# 7. Ngày mua hàng trong tuần (Thứ 2=0, Chủ Nhật=6)
df['purchase_day_of_week'] = df['order_purchase_timestamp'].dt.dayofweek

print("Đã tạo xong các feature mới. Xem thử một vài dòng:")
display(df[['review_score', 'delivery_days', 'estimated_delivery_days', 'delivery_delay', 'is_late', 'freight_ratio']].head())

Đã tạo xong các feature mới. Xem thử một vài dòng:


,review_score,delivery_days,estimated_delivery_days,delivery_delay,is_late,freight_ratio
0,5,7.614421,15.625671,-8.011250,0,0.225637
1,4,16.216181,18.546458,-2.330278,0,0.083076
2,5,7.948437,21.393391,-13.444954,0,0.089799
3,4,6.147269,11.582928,-5.435660,0,0.984603
4,5,25.114352,40.418160,-15.303808,0,0.090745


In [3]:
# Lựa chọn các cột cuối cùng cho mô hình
# Chúng ta sẽ loại bỏ các cột ID, các cột ngày tháng gốc và các cột đã dùng để tạo feature
features_to_keep = [
    # Target
    'review_score',
    
    # Numeric features
    'price',
    'freight_value',
    'product_weight_g',
    'product_volume_cm3',
    'delivery_days',
    'delivery_delay',
    'freight_ratio',
    'purchase_day_of_week',
    
    # Categorical features
    'product_category_name',
    'seller_state',
    'customer_state'
]

df_model_ready = df[features_to_keep].copy()

# Xử lý missing values (nếu có)
# Ví dụ: điền giá trị trung bình cho các cột số
for col in df_model_ready.select_dtypes(include=np.number).columns:
    if df_model_ready[col].isnull().any():
        median_val = df_model_ready[col].median()
        df_model_ready[col].fillna(median_val, inplace=True)

# Ví dụ: điền 'unknown' cho các cột category
for col in df_model_ready.select_dtypes(include='object').columns:
    if df_model_ready[col].isnull().any():
        df_model_ready[col].fillna('unknown', inplace=True)
        
print("DataFrame đã sẵn sàng cho mô hình:")
df_model_ready.info()

DataFrame đã sẵn sàng cho mô hình:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110005 entries, 0 to 110004
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   review_score           110005 non-null  int64  
 1   price                  110005 non-null  float64
 2   freight_value          110005 non-null  float64
 3   product_weight_g       110005 non-null  float64
 4   product_volume_cm3     110005 non-null  float64
 5   delivery_days          110005 non-null  float64
 6   delivery_delay         110005 non-null  float64
 7   freight_ratio          110005 non-null  float64
 8   purchase_day_of_week   110005 non-null  int32  
 9   product_category_name  110005 non-null  object 
 10  seller_state           110005 non-null  object 
 11  customer_state         110005 non-null  object 
dtypes: float64(7), int32(1), int64(1), object(3)
memory usage: 9.7+ MB


C:\Users\Windows\AppData\Local\Temp\ipykernel_16212\261649929.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_model_ready[col].fillna(median_val, inplace=True)
C:\Users\Windows\AppData\Local\Temp\ipykernel_16212\261649929.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


In [4]:
# Tạo thư mục data/processed nếu chưa có
processed_data_path = '../data/processed'
os.makedirs(processed_data_path, exist_ok=True)

# Lưu file
output_path = os.path.join(processed_data_path, 'model_ready_data.csv')
df_model_ready.to_csv(output_path, index=False)

print(f"Đã lưu bộ dữ liệu đã xử lý tại: {output_path}")

Đã lưu bộ dữ liệu đã xử lý tại: ../data/processed\model_ready_data.csv
